In [1]:
from tqdm import tqdm

tqdm.pandas()
import pandas as pd
import ast
import re
import torch
import torch.nn.functional as F
from dataset_utils import *

# Loading Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("go_emotions", "raw")
dataset.set_format(type="pandas")
df_train = dataset["train"][:]

emotions = list(df_train.columns)[9:]
columns = ["text"] + emotions
# select only texts and emotions
df_train = df_train[columns]


# create new column with list of emotions (in strings)
df_train["ds_emotions"] = df_train[emotions].progress_apply(
    lambda x: x.index[x == 1].tolist(), axis=1
)

100%|██████████| 211225/211225 [00:13<00:00, 16216.11it/s]


In [3]:
samples = df_train[df_train["ds_emotions"].apply(lambda x: "neutral" in x)]
samples = samples.sample(20, random_state=7102023)
# drop emotions columns
samples = samples.drop(emotions, axis=1)

# LLM Labeling

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

SYSTEM_MESSAGE = "You're an AI expert trained to analyze and categorize emotions present in text. Your goal is to analyze each piece of text according to instructions"

PROMPT_TEMPLATE = f"""
Given: {{comment}}
Emotion list: {emotions}
Perform an emotional analysis of the comment to by following these steps:

Literal analysis: Identify emotions directly expressed through obvious emotional words/phrases.
Contextual analysis: Consider the full context and implications to detect any additional underlying emotions.
Tonal analysis: Examine tone, subtext, and nuanced emotional cues beyond the literal language.

You may only pick one, two or three classes.

For each level of analysis, provide your findings.
Then, summarize by listing the emotion class(es) that apply in <answer> (classes) </answer>. Use "neutral" ONLY if no emotions from the list are detected.
"""


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_MESSAGE),
        # few_shot_prompt,
        ("human", PROMPT_TEMPLATE),
    ]
)

In [5]:
groq_api = ""
openai_api = ""
fireworks_api = "r8qFN4YhAZwBHev6fizSxvcny30f7Y9JuGUQwqXKdMi1N0se"

# Define models
chat1 = ChatOpenAI(
    temperature=0.6,
    model_name="accounts/fireworks/models/llama-v3-70b-instruct",
    openai_api_base="https://api.fireworks.ai/inference/v1",
    openai_api_key=fireworks_api,
)

chain1 = final_prompt | chat1 | StrOutputParser()

# chat2 = ChatOpenAI(api_key=openai_api)

# chain2 = final_prompt | chat2 | StrOutputParser()

In [6]:
samples["fireworks"] = None
# for index in range(len(samples)):
for index in tqdm(samples.index):
    comment = samples.loc[index]["text"]
    label = samples.loc[index]["ds_emotions"]
    response1 = chain1.invoke({"comment": comment})
    extracted_emotions = extract_emotions(response1)
    samples.at[index, "fireworks"] = extracted_emotions

100%|██████████| 20/20 [00:30<00:00,  1.54s/it]


In [7]:
samples["ground_truth"] = [
    ["optimism","excitement"],
    ["embarrassment"],
    ["approval","admiration"],
    ["disapproval","annoyance"],
    ["annoyance","disgust"],
    ["remorse","sadness"],
    ["disapproval"],
    ["caring"],
    ["disapproval","annoyance"],
    ["disapproval",],
    ["amusment","confusion"],
    ["neutral"],
    ["caring"],
    ["approval"],
    ["neutral"],
    ["excitement"],
    ["realization", "disappintment"],
    ["disapproval"],
    ["sadness"],
    ["neutral"],
]
gt = samples[["text", "ground_truth"]]
gt.to_csv("ground_truth.csv", index=True)
samples.to_csv("samples.csv", index = False)

In [8]:
# Sample input
samples = pd.read_csv("samples.csv")
predicted_labels = samples["fireworks"]

scores, average_bce = process_and_score(predicted_labels, samples["ground_truth"], emotions)

save_results_to_csv(scores, 'results.csv')


In [43]:
samples["fireworks_70b_prompt1_T.5"] = [['optimism', 'excitement'],
 ['amusement', 'embarrassment'],
 ['admiration', 'gratitude', 'approval'],
 ['annoyance', 'disapproval'],
 ['amusement', 'annoyance'],
 ['disappointment', 'sadness'],
 ['amusement', 'annoyance'],
 ['caring', 'relief'],
 ['annoyance', 'love'],
 ['annoyance', 'skepticism'],
 ['approval', 'annoyance'],
 ['neutral'],
 ['caring', 'optimism'],
 ['amusement'],
 ['curiosity', 'neutrality'],
 ['excitement'],
 ['disappointment', 'disapproval'],
 ['amusement', 'sarcasm'],
 ['amusement', 'optimism'],
 ['amusement', 'surprise']] 